## Parks


Gather neighborhood data to pinpoint which neighborhoods the parks are in

In [ ]:
import json
import geojson
from shapely.geometry import shape, Point

# Load GeoJSON data for neighborhoods
with open('neighborhoods.geojson') as f:
    geojson_data = json.load(f)

# Extract neighborhood polygons
polygons = []
hoods = []
for feature in geojson_data['features']:
    polygon = shape(feature['geometry'])
    polygons.append(polygon)
    hoods.append(feature['properties']['hood'])

# Define a function to check if a point is within any polygon
def is_point_within_polygon(lat, lon):
    point = Point(lon, lat)
    for i, polygon in enumerate(polygons):
        if polygon.contains(point):
            return hoods[i]
    return None

## Import the Parks csv

In [ ]:
# Read the CSV file containing latitude and longitude for parks
parks_data = []
with open('data\city-parks.csv', 'r') as file:
    next(file)  # Skip header
    for line in file:
        parts = line.strip().split(',')
        lat, lon = map(float, parts[-2:])
        parks_data.append((lat, lon))

## Count Each Park


Every time that a park is found within the latitude and longitude of the neighborhood (from the neighborhoods dataset) we add one point to the parks entry in the neighborhood dictionary.

In [ ]:
# Create a dictionary to store parks count
parks_results = {"neighborhoods": []}

# Iterate through each park point
for lat, lon in parks_data:
    hood = is_point_within_polygon(lat, lon)
    if hood is not None:
        found = False
        for entry in parks_results["neighborhoods"]:
            if entry["name"] == hood:
                entry["parks"] += 1
                found = True
                break
        if not found:
            parks_results["neighborhoods"].append({"name": hood, "parks": 1})

## Bus Stops 

For the bus stops, we had to convert the data from GeoJson to csv to make it easier to iterate through.

In [ ]:
import json
import csv

# Load GeoJSON data
with open('data\prt-stops.geojson', 'r') as f:
    data = json.load(f)

# Extract necessary information
rows = []
for index, feature in enumerate(data['features']):
    latitude = feature['properties']['Latitude']
    longitude = feature['properties']['Longitude']
    rows.append([index, latitude, longitude])

# Write to CSV
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Index', 'Latitude', 'Longitude'])
    writer.writerows(rows)

print("CSV file created successfully.")

## Counting Bus stops

From here, it is the exact same process as gathering the data from the parks csv

In [ ]:
# Read the CSV file containing latitude and longitude for prt-stops
prt_stops_data = []
with open('data\prt-stops.csv', 'r') as file:
    next(file)  # Skip header
    for line in file:
        parts = line.strip().split(',')
        lat, lon = map(float, parts[-2:])
        prt_stops_data.append((lat, lon))

# Create a dictionary to store prt-stops count
prt_stops_results = {"neighborhoods": []}

# Iterate through each prt-stop point
for lat, lon in prt_stops_data:
    hood = is_point_within_polygon(lat, lon)
    if hood is not None:
        found = False
        for entry in prt_stops_results["neighborhoods"]:
            if entry["name"] == hood:
                entry["prt-stops"] += 1
                found = True
                break
        if not found:
            prt_stops_results["neighborhoods"].append({"name": hood, "prt-stops": 1})